In [1]:
"""
    모델 및 데이터셋 클래스 정의하는 코드
"""
import torch
from torch import nn
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from math import sqrt
from datetime import datetime
import pickle

# 시계열 데이터를 처리하는 클래스를 정의합니다.
class TimeSeriesDataset(Dataset):
    def __init__(self, dataframe, seq_len=7*24, pred_len=24):
        self.seq_len = seq_len  # 입력 시퀀스의 길이를 정의합니다.
        self.pred_len = pred_len  # 예측할 시퀀스의 길이를 정의합니다.
        self.scaler = MinMaxScaler()  # 데이터 정규화를 위한 MinMaxScaler 객체를 생성합니다.

        self.dataframe = self._preprocess(dataframe)  # 데이터 전처리 함수를 호출하여 dataframe을 전처리합니다.

    def _preprocess(self, df):
        # 누락된 값을 시계열의 이전 값으로 채웁니다.
        df.fillna(method='ffill', inplace=True)

        # 숫자형 열을 [0, 1] 범위로 정규화합니다.
        numerical_cols = df.select_dtypes(include=[np.number]).columns
        df[numerical_cols] = self.scaler.fit_transform(df[numerical_cols])

        # 범주형 변수를 원-핫 인코딩합니다.
        categorical_cols = df.select_dtypes(include=['object']).columns
        if not categorical_cols.empty:
            encoder = OneHotEncoder()
            encoded = encoder.fit_transform(df[categorical_cols])
            encoded_df = pd.DataFrame(encoded.toarray(), columns=encoder.get_feature_names(categorical_cols))
            
            # 원래의 범주형 열을 삭제하고 인코딩된 열과 병합합니다.
            df.drop(columns=categorical_cols, inplace=True)
            df = pd.concat([df, encoded_df], axis=1)
        
        return df

    def __len__(self):
        return len(self.dataframe) - self.seq_len - self.pred_len + 1  # 데이터셋의 전체 길이를 반환합니다.

    def __getitem__(self, idx):
        x = self.dataframe.iloc[idx:idx+self.seq_len]  # 입력 시퀀스를 선택합니다.
        # 마지막 56열이 전력 값이라고 가정하고 예측할 시퀀스를 선택합니다.
        y = self.dataframe.iloc[idx+self.seq_len:idx+self.seq_len+self.pred_len, -56:] 
        return torch.Tensor(x.values), torch.Tensor(y.values).reshape(-1)  # y 값을 평탄화하여 반환합니다.
    
# LSTM 모델을 정의합니다.
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers):
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim  # LSTM의 은닉층의 차원을 정의합니다.
        self.n_layers = n_layers  # LSTM 층의 개수를 정의합니다.

        self.lstm = nn.LSTM(input_dim, hidden_dim, n_layers, batch_first=True)  # LSTM 층을 정의합니다.
        self.fc = nn.Linear(hidden_dim, output_dim)  # 완전 연결 층을 정의합니다.

    def forward(self, x):
        # 초기 은닉 상태와 셀 상태를 초기화합니다.
        h0 = torch.zeros(self.n_layers, x.size(0), self.hidden_dim).to(x.device)
        c0 = torch.zeros(self.n_layers, x.size(0), self.hidden_dim).to(x.device)

        # LSTM 층을 통해 데이터를 전달하고 출력을 얻습니다.
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])  # 완전 연결 층을 통해 출력을 얻습니다.
        return out

In [2]:
"""
    데이터셋 읽고 set으로 분할하는 코드
"""
# pandas 라이브러리를 사용하여 엑셀 파일을 불러옵니다.
df = pd.read_excel('/home/kimyirum/EMS/ict-2023-ems/load/data/merged_data_KW.xlsx')

# TimeSeriesDataset 클래스의 인스턴스를 생성합니다. 위에서 정의한 클래스를 사용하여 데이터를 전처리합니다.
dataset = TimeSeriesDataset(df)

# 학습, 검증 및 테스트 데이터 세트의 크기를 정의합니다.
train_size = int(0.7 * len(dataset))  # 전체 데이터의 70%를 학습 데이터로 사용
val_size = int(0.2 * len(dataset))    # 전체 데이터의 20%를 검증 데이터로 사용
test_size = len(dataset) - train_size - val_size  # 나머지 데이터를 테스트 데이터로 사용

# 전체 데이터셋을 학습, 검증 및 테스트 데이터 세트로 분할합니다.
train_set, val_set, test_set = random_split(dataset, [train_size, val_size, test_size])

In [12]:
"""
    data loader, model 초기화하는 코드
    epoch만큼 train 실행하는 코드
    모델 저장하는 코드
"""
########################################## Hyperparameters ##########################################
input_dim = len(train_set[0][0][0]) # 63
output_dim = 24*56
hidden_dim = 128
n_layers = 7
learning_rate = 0.001
num_epochs = 200
batch_size = 128
use_thread = True
patience = 20 # number of epochs to wait before stopping
########################################################################################################

# 결과를 저장할 폴더의 경로를 지정합니다.
results_folder = "/home/kimyirum/EMS/ict-2023-ems/load/results/"
# 현재의 시간 정보를 가져옵니다.
now = datetime.now()
# 현재 시간 정보를 문자열 포맷으로 변환합니다. (예: 20230807_143000)
now_str = now.strftime('%Y%m%d_%H%M%S')
# 결과 메트릭을 저장할 파일의 이름을 지정합니다.
filename_metrics = f'{now_str}.pkl'
# 학습된 모델을 저장할 파일의 이름을 지정합니다.
filename_model = f'model_{now_str}.pt'

# DataLoader
# use_thread 변수의 값에 따라 DataLoader의 num_workers 값을 설정합니다.
if use_thread:
    train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True, num_workers=8)
    val_loader = DataLoader(dataset=val_set, batch_size=batch_size, shuffle=False, num_workers=8)
    test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=False, num_workers=8)
else:
    train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(dataset=val_set, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=False)

# LSTM 모델, 손실 함수, 최적화 도구를 초기화합니다.
model = LSTMModel(input_dim, hidden_dim, output_dim, n_layers)
criterion = nn.MSELoss()  # MSE 손실 함수를 사용합니다.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  # Adam 최적화 도구를 사용합니다.

# 초기 검증 손실을 무한대로 설정합니다.
best_val_loss = float("inf")
no_improve_epoch = 0

########################################## Training loop ##########################################
# 주어진 에포크만큼 모델을 학습합니다.
for epoch in range(num_epochs):
    model.train()
    for batch_idx, (data, targets) in enumerate(train_loader):
        # 순전파를 수행합니다.
        outputs = model(data)
        loss = criterion(outputs, targets)

        # 역전파 및 최적화를 수행합니다.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # 모델을 검증합니다.
    model.eval()
    with torch.no_grad():
        val_losses = []
        for data, targets in val_loader:
            outputs = model(data)
            loss = criterion(outputs, targets)
            val_losses.append(loss.item())
        avg_val_loss = sum(val_losses) / len(val_losses)
        print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {avg_val_loss:.4f}', end='')
        
        # 검증 손실이 감소했을 경우, 모델을 저장합니다.
        if avg_val_loss < best_val_loss:
            print(", Save model")
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), results_folder+filename_model)
            no_improve_epoch = 0
        else:
            print("")
            no_improve_epoch += 1
            
        # 조기 종료 조건: 검증 손실이 연속으로 patience동안 개선되지 않을 때 학습을 중단합니다.
        if no_improve_epoch > patience:
            print('Early stopping...')
            break
########################################################################################################

########################################## Evaluate the model ##########################################
# 모델을 평가 모드로 설정합니다. 이는 dropout, batch normalization 등의 레이어가 
# 학습 모드와 다르게 작동해야 할 때 필요합니다.
model.eval()

# torch.no_grad()를 사용하여 autograd의 gradient 계산을 비활성화합니다. 
# 이렇게 하면 메모리 사용량을 줄이고 속도를 높일 수 있습니다.
with torch.no_grad():
    all_targets = []  # 실제 목표 값들을 저장할 리스트를 초기화합니다.
    all_outputs = []  # 모델의 예측 값을 저장할 리스트를 초기화합니다.
    for data, targets in test_loader:  # 테스트 데이터로더에서 배치를 반복적으로 가져옵니다.
        outputs = model(data)  # 모델을 사용하여 입력 데이터에 대한 예측값을 생성합니다.
        all_targets.append(targets.numpy())  # 목표 값을 리스트에 추가합니다.
        all_outputs.append(outputs.numpy())  # 예측 값을 리스트에 추가합니다.

# 목표 값과 예측 값을 모두 단일 넘파이 배열로 연결(flatten)합니다.
all_targets = np.concatenate(all_targets).flatten()
all_outputs = np.concatenate(all_outputs).flatten()

# 평균 절대 오차(MAE), 평균 제곱 오차(MSE) 및 제곱근 평균 제곱 오차(RMSE)를 계산합니다.
mae = mean_absolute_error(all_targets, all_outputs)
mse = mean_squared_error(all_targets, all_outputs)
rmse = sqrt(mse)

# 계산된 지표들을 출력합니다. 이 값들은 모델의 성능을 평가하는 데 사용됩니다.
print(f'MAE: {mae:.4f}, MSE: {mse:.4f}, RMSE: {rmse:.4f}')

########################################################################################################


Epoch [1/200], Validation Loss: 0.1055, Save model
Epoch [2/200], Validation Loss: 0.0520, Save model
Epoch [3/200], Validation Loss: 0.0471, Save model
Epoch [4/200], Validation Loss: 0.0449, Save model
Epoch [5/200], Validation Loss: 0.0438, Save model
Epoch [6/200], Validation Loss: 0.0434, Save model
Epoch [7/200], Validation Loss: 0.0434
Epoch [8/200], Validation Loss: 0.0426, Save model
Epoch [9/200], Validation Loss: 0.0429
Epoch [10/200], Validation Loss: 0.0432
Epoch [11/200], Validation Loss: 0.0431
Epoch [12/200], Validation Loss: 0.0418, Save model
Epoch [13/200], Validation Loss: 0.0414, Save model
Epoch [14/200], Validation Loss: 0.0410, Save model
Epoch [15/200], Validation Loss: 0.0411
Epoch [16/200], Validation Loss: 0.0401, Save model
Epoch [17/200], Validation Loss: 0.0419
Epoch [18/200], Validation Loss: 0.0400, Save model
Epoch [19/200], Validation Loss: 0.0406
Epoch [20/200], Validation Loss: 0.0397, Save model
Epoch [21/200], Validation Loss: 0.0398
Epoch [22/200

In [13]:
"""
    위에서 학습시킨 모델의 성능이 괜찮다면, 파라미터 정보를 pkl 파일로 저장하는 코드
"""

# 사용한 하이퍼파라미터들을 저장합니다.
hyperparams = {
    'learning_rate': learning_rate,
    'batch_size': batch_size,
    'num_epochs': num_epochs,
    'hidden_dim': hidden_dim,
    'n_layers': n_layers
}

# 데이터 정규화에 사용된 scaler를 저장합니다.
scalers = dataset.scaler

# 성능 지표를 저장합니다.
metrics = {
    'MAE': mae,
    'MSE': mse,
    'RMSE': rmse,
}

# 위에서 정의한 모든 결과를 하나의 사전에 합칩니다.
results = {
    'Hyperparameters': hyperparams,
    'Scalers': scalers,
    'Metrics': metrics
}

# 결합된 결과를 pickle 파일로 저장합니다.
with open(results_folder + filename_metrics, 'wb') as f:
    pickle.dump(results, f)

# 테스트를 위해
# pickle 파일로부터 결과를 불러옵니다.
with open(results_folder + filename_metrics, 'rb') as f:
    loaded_results = pickle.load(f)

# 불러온 결과에서 데이터에 접근할 수 있습니다.
print(loaded_results['Hyperparameters'])
print(loaded_results['Scalers'])
print(loaded_results['Metrics'])


{'learning_rate': 0.001, 'batch_size': 128, 'num_epochs': 200, 'hidden_dim': 128, 'n_layers': 7}
MinMaxScaler()
{'MAE': 0.05801902, 'MSE': 0.007335685, 'RMSE': 0.08564861466211036}


In [14]:
""" 
    테스트 세트의 첫 번째 시퀀스에 대한 예측을 수행한 후, 예측된 값과 실제 목표값 사이의 차이를 계산하는 코드
    이러한 차이를 기반으로 여러 성능 지표를 계산하며, 전체 에러를 출력하는 코드
    *df는 원래의 데이터셋이라 가정하며 'date' 컬럼을 포함한다고 가정합니다.
"""
# 건물 이름을 가져옵니다.
building_names = df.columns[-56:]  # 필요에 따라 이 값을 조절하세요.
# 테스트 세트를 위한 DataLoader를 생성합니다.
test_loader = DataLoader(dataset=test_set, batch_size=1, shuffle=False)
# 테스트 세트에서 첫 번째 시퀀스와 그 목표값을 가져옵니다.
real_sequence, real_target = next(iter(test_loader))

# 모델을 평가 모드로 전환합니다.
model.eval()

# 예측을 수행합니다.
with torch.no_grad():
    prediction = model(real_sequence)

print(prediction.shape, real_target.shape)
prediction = prediction.squeeze(0).reshape(24, 56).numpy()
real_target = real_target.view(24, 56).numpy()

# 패딩을 추가합니다.
padding = np.zeros((prediction.shape[0], 7))
prediction_pad = np.hstack((padding, prediction))
real_target_pad = np.hstack((padding, real_target))
# print(prediction_pad.shape, real_target_pad.shape)

# 역변환을 적용하여 정규화를 해제합니다.
prediction_inv = dataset.scaler.inverse_transform(prediction_pad)
real_target_inv = dataset.scaler.inverse_transform(real_target_pad)

# 처음 7개의 컬럼을 삭제합니다.
prediction_inv = np.delete(prediction_inv, np.s_[:7], axis=1)
real_target_inv = np.delete(real_target_inv, np.s_[:7], axis=1)

# 원래의 형태로 다시 변형합니다.
prediction = prediction_inv.reshape(prediction.shape)
real_target = real_target_inv.reshape(real_target.shape)

# 에러(실제 목표값과 예측값의 차이)를 계산합니다.
error = real_target - prediction

# 예측값을 위한 DataFrame을 생성합니다.
predicted_df = pd.DataFrame(prediction, columns=building_names)
real_target_df = pd.DataFrame(real_target, columns=building_names)
error_df = pd.DataFrame(error, columns=building_names)
err_total = total = error_df.values.flatten().sum()

# print("Predicted Values for next 24 hours:")
# print(predicted_df)

# print("Real Values for next 24 hours:")
# print(real_target_df)

print("Error for next 24 hours:")
print(error_df)

# 성능 지표를 계산합니다.
mae = mean_absolute_error(real_target, prediction)
mse = mean_squared_error(real_target, prediction)
rmse = sqrt(mse)
print(f'MAE: {mae:.4f}, MSE: {mse:.4f}, RMSE: {rmse:.4f} (no normalization)')
print("err_total: ", err_total)


torch.Size([1, 1344]) torch.Size([1, 1344])
Error for next 24 hours:
        0_SV-2      1_SV-5    2_SV-6      3_SV-7  4_HV-NM1    5_HV-NM2  \
0    55.119337   54.357242 -0.006374   45.354768  0.006537   63.761941   
1    55.774660   25.734787  0.002648   16.815664 -0.007302   26.146778   
2    12.868810   -2.931396 -0.006189   -4.936558  0.001629   22.293055   
3    83.161757   59.323946 -0.006972    6.364716  0.013761  -34.267422   
4   -19.356127   -3.996244  0.000817   -3.143047  0.002588   58.776626   
5   -41.951238  -50.216364 -0.007442  -27.290220  0.001287  -18.936129   
6    10.442949   52.436289 -0.004823   31.854352  0.015007   36.703437   
7   -10.491514   -8.881780 -0.006200   14.832201 -0.012433   -7.354283   
8     2.229872  -10.735801 -0.001054  -13.933305  0.000888  -15.914083   
9   -67.290851  -78.290749 -0.017405  -54.689699 -0.000209  -67.667446   
10  -79.108715  -49.386932 -0.004426  -59.041306  0.003913  -74.605780   
11   41.941002   18.296055  0.005105   12.3

In [ ]:
"""
    최적의 하이퍼파라미터를 찾기 위해 optuna 라이브러리를 이용해서 실험하는 코드
    *실험 log는 results/log_optuna.txt로 저장함
"""
# Optuna를 사용할 것인지 여부를 결정하는 플래그
do_optuna = False

# optuna 관련 라이브러리를 가져옵니다.
import optuna
import optuna.logging

# Optuna의 기본 로깅 핸들러를 활성화합니다.
optuna.logging.enable_default_handler()

# 로깅의 상세 수준을 설정합니다.
optuna.logging.set_verbosity(optuna.logging.INFO)

# Optuna를 사용하여 최적화할 목적 함수를 정의합니다.
def objective(trial):
    # Optuna를 사용하여 하이퍼파라미터를 추정합니다.
    lr = trial.suggest_float('lr', 1e-5, 1e-1, log=True)
    hidden_dim = trial.suggest_int('hidden_dim', 50, 300)
    n_layers = trial.suggest_int('n_layers', 5, 9)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
    
    # 데이터 로더를 설정합니다.
    if use_thread:
        train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True, num_workers=8)
        val_loader = DataLoader(dataset=val_set, batch_size=batch_size, shuffle=False, num_workers=8)
        test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=False, num_workers=8)
    else:
        train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(dataset=val_set, batch_size=batch_size, shuffle=False)
        test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=False)
    
    # 모델, 손실 함수, 최적화 알고리즘을 초기화합니다.
    model = LSTMModel(input_dim, hidden_dim, output_dim, n_layers)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    best_val_loss = float("inf") # 초기에는 무한대로 설정합니다.
    no_improve_epoch = 0

    # 훈련 루프입니다.
    for epoch in range(num_epochs):
        model.train()
        for batch_idx, (data, targets) in enumerate(train_loader):
            # 순전파
            outputs = model(data)
            loss = criterion(outputs, targets)

            # 역전파 및 최적화
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # 모델을 검증합니다.
        model.eval()
        with torch.no_grad():
            val_losses = []
            for data, targets in val_loader:
                outputs = model(data)
                loss = criterion(outputs, targets)
                val_losses.append(loss.item())
            avg_val_loss = sum(val_losses) / len(val_losses)
            
            # 검증 손실이 줄어들면 모델을 저장합니다.
            if avg_val_loss < best_val_loss:
                best_val_loss = avg_val_loss
                no_improve_epoch = 0
            else:
                no_improve_epoch += 1
                
            # 일찍 중단하기 위한 조건
            if no_improve_epoch > patience:
                print('Early stopping...')
                break

    return best_val_loss

# do_optuna 플래그가 True로 설정되어 있으면 Optuna를 사용하여 하이퍼파라미터를 최적화합니다.
if do_optuna:
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=100, n_jobs=4)

    print("Best trial:")
    trial = study.best_trial

    print(" Value: ", trial.value)

    print(" Params: ")
    for key, value in trial.params.items():
        print(f"    {key}: {value}")
